In [1]:
import pandas as pd
from keras.models import Sequential
from scipy.ndimage.interpolation import shift
from keras import optimizers
from keras.layers import Dense, Activation, Dropout
from keras.layers import Conv1D, GlobalAveragePooling1D, MaxPooling1D
from keras.utils import np_utils
import numpy as np
from scipy.sparse import csr_matrix, hstack
from sklearn.model_selection import train_test_split
import re
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import confusion_matrix, classification_report
from sklearn import preprocessing

Using TensorFlow backend.


In [2]:
data = pd.read_csv('data.csv')
print(data.shape)

(2554, 2)


In [3]:
t = np.array([0,1,2,3,3,2,1,0]) 
print(np.gradient(t))
print(np.gradient(t,2))

[ 1.   1.   1.   0.5 -0.5 -1.  -1.  -1. ]
[ 0.5   0.5   0.5   0.25 -0.25 -0.5  -0.5  -0.5 ]


In [4]:
Y = data.as_matrix(columns=['currency']).reshape(-1)
Y = np.gradient(Y, 2)
#Y = np.gradient(Y)

In [5]:
num_classes = None

In [6]:
mutli_cls = 1
if mutli_cls ==0:
    Y = np.where(Y > 0, 1, 0)
    num_classes = 2
    f = np.bincount(Y)
    print(f)
else:
    num_classes = 3
    offset = 1e-3
    Y[np.where(Y >= offset)] = 2
    Y[np.where((Y > -offset) & (Y < offset))] = 1
    Y[np.where(Y <= -offset)] = 0
    Y = Y.astype('int64')
    f = np.bincount(Y)
    print(f)
    print(1-((f[0]+f[2])/np.sum(f)))
Y = shift(Y, -2, cval=1)

[ 755 1078  721]
0.422083007048


In [7]:
Y = np_utils.to_categorical(Y, num_classes)
Y

array([[ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       ..., 
       [ 0.,  1.,  0.],
       [ 0.,  1.,  0.],
       [ 0.,  1.,  0.]])

In [8]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(data['name'].tolist())
X

<2554x35799 sparse matrix of type '<class 'numpy.int64'>'
	with 797524 stored elements in Compressed Sparse Row format>

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, train_size=0.8)
X_train = X_train.toarray()
X_test = X_test.toarray()

In [123]:
model = Sequential()

model.add(Dense(1000, activation='relu', input_shape=(X.shape[1],)))
model.add(Dropout(0.75))
model.add(Dense(200, activation='softmax'))
model.add(Dropout(0.5))
model.add(Dense(200, activation='softmax'))
model.add(Dropout(0.5))
model.add(Dense(200, activation='softmax'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

In [124]:
model.compile(optimizer='RMSprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_36 (Dense)             (None, 1000)              35800000  
_________________________________________________________________
dropout_29 (Dropout)         (None, 1000)              0         
_________________________________________________________________
dense_37 (Dense)             (None, 200)               200200    
_________________________________________________________________
dropout_30 (Dropout)         (None, 200)               0         
_________________________________________________________________
dense_38 (Dense)             (None, 200)               40200     
_________________________________________________________________
dropout_31 (Dropout)         (None, 200)               0         
_________________________________________________________________
dense_39 (Dense)             (None, 200)               40200     
__________

In [125]:
# history = model.fit(X_train.toarray(), 
#                     y_train, 
#                     epochs=20, 
#                     batch_size=128)
# score = model.evaluate(X_test.toarray(), 
#                y_test, 
#                batch_size=128)
# print('Test score:', score[0])
# print('Test accuracy:', score[1])

In [126]:
history = model.fit(X_train, y_train, 
                    epochs=10,
                    validation_data=(X_test, y_test),
                    batch_size=128)

Train on 2043 samples, validate on 511 samples
Epoch 1/10
2043/2043 [==============================] - 1s - loss: 1.0956 - acc: 0.4072 - val_loss: 1.0939 - val_acc: 0.4090
Epoch 2/10
2043/2043 [==============================] - 0s - loss: 1.0914 - acc: 0.4258 - val_loss: 1.0913 - val_acc: 0.4090
Epoch 3/10
2043/2043 [==============================] - 0s - loss: 1.0882 - acc: 0.4258 - val_loss: 1.0895 - val_acc: 0.4090
Epoch 4/10
2043/2043 [==============================] - 0s - loss: 1.0860 - acc: 0.4258 - val_loss: 1.0881 - val_acc: 0.4090
Epoch 5/10
2043/2043 [==============================] - 0s - loss: 1.0842 - acc: 0.4258 - val_loss: 1.0872 - val_acc: 0.4090
Epoch 6/10
2043/2043 [==============================] - 0s - loss: 1.0831 - acc: 0.4258 - val_loss: 1.0866 - val_acc: 0.4090
Epoch 7/10
2043/2043 [==============================] - 0s - loss: 1.0819 - acc: 0.4258 - val_loss: 1.0862 - val_acc: 0.4090
Epoch 8/10
2043/2043 [==============================] - 0s - loss: 1.0811 - ac

In [127]:
pred = np.argmax(model.predict(X_test), axis=1)
true = np.argmax(y_test, axis=1)
if mutli_cls==0:
    print(confusion_matrix(true, pred, labels=[0,1]))
    target_names = ['down','up']
    print(classification_report(true, pred, target_names=target_names))
else:
    print(confusion_matrix(true, pred, labels=[0,1,2]))
    target_names = ['down','non','up']
    print(classification_report(true, pred, target_names=target_names))

[[  0 155   0]
 [  0 209   0]
 [  0 147   0]]
             precision    recall  f1-score   support

       down       0.00      0.00      0.00       155
        non       0.41      1.00      0.58       209
         up       0.00      0.00      0.00       147

avg / total       0.17      0.41      0.24       511



/usr/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
